In [1]:
!curl -O https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 80.2M  100 80.2M    0     0  8217k      0  0:00:09  0:00:09 --:--:-- 16.1M


In [4]:
!export TF_ENABLE_ONEDNN_OPTS=0

In [ ]:
!tar -xf aclImdb_v1.tar.gz

In [3]:
!rm -r aclImdb/train/unsup

In [4]:
!cat aclImdb/train/pos/4077_10.txt

I first saw this back in the early 90s on UK TV, i did like it then but i missed the chance to tape it, many years passed but the film always stuck with me and i lost hope of seeing it TV again, the main thing that stuck with me was the end, the hole castle part really touched me, its easy to watch, has a great story, great music, the list goes on and on, its OK me saying how good it is but everyone will take there own best bits away with them once they have seen it, yes the animation is top notch and beautiful to watch, it does show its age in a very few parts but that has now become part of it beauty, i am so glad it has came out on DVD as it is one of my top 10 films of all time. Buy it or rent it just see it, best viewing is at night alone with drink and food in reach so you don't have to stop the film.<br /><br />Enjoy

## 20k files for training, 5k files for validation and 25k files for testing

In [5]:
import os, pathlib, shutil, random\

base_dir = pathlib.Path("aclImdb")
val_dir = base_dir / "val"
train_dir = base_dir / "train"

for category in ("neg", "pos"):
    os.makedirs(val_dir / category)
    files = os.listdir(train_dir / category)
    random.Random(1337).shuffle(files)
    num_val_samples = int(0.2 * len(files))
    val_files = files[-num_val_samples:]
    for fname in val_files:
        shutil.move(train_dir / category / fname,
            val_dir / category / fname)

In [ ]:
from tensorflow import keras

batch_size = 32

train_ds = keras.utils.text_dataset_from_directory(
"aclImdb/train", batch_size=batch_size
)
val_ds = keras.utils.text_dataset_from_directory(
"aclImdb/val", batch_size=batch_size
)
test_ds = keras.utils.text_dataset_from_directory(
"aclImdb/test", batch_size=batch_size
)

# Data Characteristics

In [3]:
for inputs, targets in train_ds:
    print("inputs.shape:", inputs.shape)
    print("inputs.dtype:", inputs.dtype)
    print("targets.shape:", targets.shape)
    print("targets.dtype:", targets.dtype)
    print("inputs[0]:", inputs[0])
    print("targets[0]:", targets[0])
    break

inputs.shape: (32,)
inputs.dtype: <dtype: 'string'>
targets.shape: (32,)
targets.dtype: <dtype: 'int32'>
inputs[0]: tf.Tensor(b"Man With the Gun is pretty much forgotten now, but caused a minor storm of media interest back in 1955 when Robert Mitchum turned down both Jett Rink in Giant (which had actually been written for him and which was subsequently substantially reworked) and Charles Laughton's intended version of The Naked and the Dead to make it instead. Despite some obvious production problems and some harsh lighting that occasionally renders both Mitch and Jan Sterling in unflattering tones, it's a terrific dark western that more than stands comparison with his earlier Blood on the Moon as his 'town tamer' sets to work on a town that never had the chance to grow up before getting run down by the local badmen before turning out to \xc2\x96 possibly \xc2\x96 be almost as bad as the men he dispatches. Certainly his way of dealing with news of a death in the family \xc2\x96 burning

In [12]:
import os
from collections import Counter
import re
from nltk.util import ngrams


directory = "aclImdb/train/neg"

# Words to filter n-grams



phrase_freq = Counter()


def clean_text(text):
    cleaned_text = re.sub(r'\bbr\b', '', text)
    cleaned_word_list = re.findall(r'\b\w+\b', cleaned_text.lower())
    return cleaned_word_list


def contains_filter_word(phrase):
    filter_phrases = ["worst", "waste", "time wasting", "racist","crap", "disappointed", "so bad", "don't watch"]
    for filter_phrase in filter_phrases:
        if filter_phrase in ' '.join(phrase):
            return True
    return False


for filename in os.listdir(directory):
    with open(os.path.join(directory, filename), 'r', encoding='utf-8') as file:
        text = file.read()
        words = clean_text(text)
        # Generate n-grams and update phrase frequencies
        n_grams = ngrams(words, 6)
        filtered_ngrams = filter(contains_filter_word, n_grams)
        phrase_freq.update(filtered_ngrams)

# Print the most common phrases
#print("Most common filtered phrases:")
#for phrase, freq in phrase_freq.most_common(50):
#    print(f"{' '.join(phrase)}: {freq}")

top_phrases = [' '.join(phrase) for phrase, _ in phrase_freq.most_common(30)]
print(top_phrases)


['one of the worst movies i', 'worst movie i have ever seen', 'the worst movie i have ever', 'this is one of the worst', 'of the worst movies i have', 'the worst movie i ve ever', 'worst movie i ve ever seen', 'the worst movies i have ever', 'worst movies i have ever seen', 'one of the worst films i', 'to be one of the worst', 'of the worst movies i ve', 'is one of the worst movies', 'one of the worst movies ever', 'worst movies i ve ever seen', 'don t waste your time or', 'don t waste your time on', 'the worst movies i ve ever', 'the worst film i have ever', 'don t waste your time with', 'worst film i have ever seen', 'of the worst films i have', 't waste your time on this', 'worst film i ve ever seen', 't waste your time or money', 'of the worst movies ever made', 't waste your time with this', 'don t waste your time watching', 'worst films i have ever seen', 'the worst film i ve ever']


# Bag-of-words approach: Bigram 

In [18]:
from tensorflow import keras
from tensorflow.keras import layers

def get_model(max_tokens=20000, hidden_dim=16):
    inputs = keras.Input(shape=(max_tokens,))
    x = layers.Dense(hidden_dim, activation="relu")(inputs)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(1, activation="sigmoid")(x)
    model = keras.Model(inputs, outputs)
    model.compile(optimizer="rmsprop",
        loss="binary_crossentropy",
        metrics=["accuracy"])
    return model

In [16]:
text_vectorization = TextVectorization(
ngrams=2,
max_tokens=20000,
output_mode="multi_hot",
)

In [19]:
text_vectorization.adapt(text_only_train_ds)

binary_2gram_train_ds = train_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)
binary_2gram_val_ds = val_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)
binary_2gram_test_ds = test_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)


Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 20000)]           0         
                                                                 
 dense_1 (Dense)             (None, 16)                320016    
                                                                 
 dropout_1 (Dropout)         (None, 16)                0         
                                                                 
 dense_2 (Dense)             (None, 1)                 17        
                                                                 
Total params: 320033 (1.22 MB)
Trainable params: 320033 (1.22 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [20]:
model = get_model()
model.summary()
callbacks = [
    keras.callbacks.ModelCheckpoint("binary_2gram.keras",
    save_best_only=True)
]
model.fit(binary_2gram_train_ds.cache(),
validation_data=binary_2gram_val_ds.cache(),
epochs=10,
callbacks=callbacks)
model = keras.models.load_model("binary_2gram.keras")
print(f"Test acc: {model.evaluate(binary_2gram_test_ds)[1]:.3f}")

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 20000)]           0         
                                                                 
 dense_3 (Dense)             (None, 16)                320016    
                                                                 
 dropout_2 (Dropout)         (None, 16)                0         
                                                                 
 dense_4 (Dense)             (None, 1)                 17        
                                                                 
Total params: 320033 (1.22 MB)
Trainable params: 320033 (1.22 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/10
625/625 [==============================] - 16s 24ms/step - loss: 0.3679 - accuracy: 0.8525 - val_loss: 0.2818 - val_accuracy: 0.8878
Epoch 2/10


# Testing

In [23]:
inputs = keras.Input(shape=(1,), dtype="string")
processed_inputs = text_vectorization(inputs)
outputs = model(processed_inputs)
inference_model = keras.Model(inputs, outputs)

import tensorflow as tf

raw_text_data = tf.convert_to_tensor([
["That was an excellent movie, I loved it."],
])
predictions = inference_model(raw_text_data)
print(f"{float(predictions[0] * 100):.2f} percent positive")

96.95 percent positive


# Sequence model: using word embeddings instead of one-hot encoding

In [13]:
from tensorflow.keras import layers

max_length = 600
max_tokens = 20000
text_vectorization = layers.TextVectorization(
max_tokens=max_tokens,
output_mode="int",
output_sequence_length=max_length,
)
text_vectorization.adapt(text_only_train_ds)
int_train_ds = train_ds.map(
lambda x, y: (text_vectorization(x), y),
num_parallel_calls=4)
int_val_ds = val_ds.map(
lambda x, y: (text_vectorization(x), y),
num_parallel_calls=4)
int_test_ds = test_ds.map(
lambda x, y: (text_vectorization(x), y),
num_parallel_calls=4)

In [14]:
from tensorflow.keras import layers

inputs = keras.Input(shape=(None,), dtype="int64")
embedded = layers.Embedding(
input_dim=max_tokens, output_dim=256, mask_zero=True)(inputs)
x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop",
loss="binary_crossentropy",
metrics=["accuracy"])
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding (Embedding)       (None, None, 256)         5120000   
                                                                 
 bidirectional (Bidirection  (None, 64)                73984     
 al)                                                             
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense (Dense)               (None, 1)                 65        
                                                                 
Total params: 5194049 (19.81 MB)
Trainable params: 5194049 (19.81 MB)
Non-trainable params: 0 (0.00 Byte)
_____________________

In [15]:
callbacks = [
keras.callbacks.ModelCheckpoint("embeddings_bidir_gru_with_masking.keras",
save_best_only=True)
]
model.fit(int_train_ds, validation_data=int_val_ds, epochs=10,
callbacks=callbacks)
model = keras.models.load_model("embeddings_bidir_gru_with_masking.keras")
print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")

Epoch 1/10


2024-02-23 15:56:03.425827: W tensorflow/core/common_runtime/type_inference.cc:339] Type inference failed. This indicates an invalid graph that escaped type checking. Error message: INVALID_ARGUMENT: expected compatible input types, but input 1:
type_id: TFT_OPTIONAL
args {
  type_id: TFT_PRODUCT
  args {
    type_id: TFT_TENSOR
    args {
      type_id: TFT_INT32
    }
  }
}
 is neither a subtype nor a supertype of the combined inputs preceding it:
type_id: TFT_OPTIONAL
args {
  type_id: TFT_PRODUCT
  args {
    type_id: TFT_TENSOR
    args {
      type_id: TFT_FLOAT
    }
  }
}

	for Tuple type infernce function 0
	while inferring type of node 'cond_36/output/_23'
2024-02-23 15:56:04.570002: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8906
2024-02-23 15:56:05.702664: I external/local_xla/xla/service/service.cc:168] XLA service 0x7f68ec774dc0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-02-23 15:56

625/625 [==============================] - 72s 103ms/step - loss: 0.4549 - accuracy: 0.7834 - val_loss: 0.3319 - val_accuracy: 0.8580
Epoch 2/10
625/625 [==============================] - 50s 81ms/step - loss: 0.2891 - accuracy: 0.8852 - val_loss: 0.3305 - val_accuracy: 0.8696
Epoch 3/10
625/625 [==============================] - 48s 76ms/step - loss: 0.2157 - accuracy: 0.9196 - val_loss: 0.3026 - val_accuracy: 0.8718
Epoch 4/10
625/625 [==============================] - 47s 75ms/step - loss: 0.1650 - accuracy: 0.9407 - val_loss: 0.3976 - val_accuracy: 0.8626
Epoch 5/10
625/625 [==============================] - 47s 75ms/step - loss: 0.1242 - accuracy: 0.9557 - val_loss: 0.3674 - val_accuracy: 0.8756
Epoch 6/10
625/625 [==============================] - 47s 75ms/step - loss: 0.0932 - accuracy: 0.9667 - val_loss: 0.3843 - val_accuracy: 0.8672
Epoch 7/10
625/625 [==============================] - 47s 75ms/step - loss: 0.0724 - accuracy: 0.9763 - val_loss: 0.5497 - val_accuracy: 0.8638
Ep

# Transformer Encoder with position aware Positional Embedding

In [24]:
from tensorflow.keras import layers
class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention = layers.MultiHeadAttention(
        num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation="relu"),
            layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization(epsilon=1e-7)
        self.layernorm_2 = layers.LayerNormalization(epsilon=1e-7)
    def call(self, inputs, mask=None):
        if mask is not None:
            mask = mask[:, tf.newaxis, :]
        attention_output = self.attention(
            inputs, inputs, attention_mask=mask)
        proj_input = self.layernorm_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)
    def get_config(self):
        config = super().get_config()
        config.update({
        "embed_dim": self.embed_dim,
        "num_heads": self.num_heads,
        "dense_dim": self.dense_dim,
        })
        return config

In [25]:
class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, input_dim, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.token_embeddings = layers.Embedding(
            input_dim=input_dim, output_dim=output_dim)
        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=output_dim)
        self.sequence_length = sequence_length
        self.input_dim = input_dim
        self.output_dim = output_dim
        
    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions
        
    def compute_mask(self, inputs, mask=None):
        return tf.math.not_equal(inputs, 0)
        
    def get_config(self):
        config = super().get_config()
        config.update({
            "output_dim": self.output_dim,
            "sequence_length": self.sequence_length,
            "input_dim": self.input_dim,
        })
        return config

In [ ]:
import tensorflow as tf

vocab_size = 20000
sequence_length = 600
embed_dim = 256
num_heads = 2
dense_dim = 32

inputs = keras.Input(shape=(None,), dtype="int64")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(inputs)
x = TransformerEncoder(embed_dim, dense_dim, num_heads)(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop",
loss="binary_crossentropy",
metrics=["accuracy"])
model.summary()

callbacks = [
keras.callbacks.ModelCheckpoint("full_transformer_encoder.keras",
save_best_only=True)
]

model.fit(int_train_ds, validation_data=int_val_ds, epochs=20,
callbacks=callbacks)
model = keras.models.load_model(
"full_transformer_encoder.keras",
custom_objects={"TransformerEncoder": TransformerEncoder,
"PositionalEmbedding": PositionalEmbedding})
print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")

Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_10 (InputLayer)       [(None, None)]            0         
                                                                 
 positional_embedding (Posi  (None, None, 256)         5273600   
 tionalEmbedding)                                                
                                                                 
 transformer_encoder (Trans  (None, None, 256)         543776    
 formerEncoder)                                                  
                                                                 
 global_max_pooling1d (Glob  (None, 256)               0         
 alMaxPooling1D)                                                 
                                                                 
 dropout_4 (Dropout)         (None, 256)               0         
                                                           